In [ ]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Lambda
from keras.optimizers import Adam, RMSprop
import sklearn.preprocessing
import sys
if "../" not in sys.path:
    sys.path.append("../") 
from lib import plotting
from PIL import Image


ENV_NAME = "Breakout-v0"
env = gym.make(ENV_NAME)

# fuentes:
# https://github.com/rohitgirdhar/Deep-Q-Networks/
# https://github.com/keon/deep-q-learning/blob/master/dqn.py
# https://github.com/AdamStelmaszczyk/dqn/

In [ ]:
class Estimator():
    """
    Red Neuronal que aproxima la función de valor estado acción (q-function)
    """   
    def __init__(self, env, frame_history=4, learning_rate=0.001):        
        n_actions = env.action_space.n
        obs_shape = (84,84,frame_history)
        model = Sequential()
        model.add(Lambda(lambda x: x / 255.0))
        
        #se define una red convolucional como en el paper: "Playing Atari with Deep Reinforcement Learning"
        model.add(Conv2D(16, 8, strides=(4, 4),activation='relu',input_shape=obs_shape))
        model.add(Conv2D(32, 4, strides=(2, 2),activation='relu'))
        model.add(Flatten())
        model.add(Dense(256, activation='relu'))
        model.add(Dense(n_actions, activation=None))
        
        # para la red convolucional como la utilizada en el paper de nature de DeepMind descomentar
        # y comentar la red anterior
        # "Human-level control through deep reinforcement learning"
        # (https://storage.googleapis.com/deepmind-media/dqn/DQNNaturePaper.pdf)
        #model = Sequential()
        #model.add(Lambda(lambda x: x / 255.0))
        #model.add(Conv2D(32, 8, strides=(4, 4),activation='relu',input_shape=obs_shape))
        #model.add(Conv2D(64, 4, strides=(2, 2),activation='relu'))
        #model.add(Conv2D(64, 3, strides=(1, 1),activation='relu'))
        #model.add(Flatten())
        #model.add(Dense(512, activation='relu'))
        #model.add(Dense(n_actions, activation=None))
        
        model.compile(loss="mse", optimizer=RMSprop(learning_rate, rho=0.95, epsilon=0.01))
        self.model = model
     
    def update(self, states, q_values, verbose=0):
        """
        Realiza un update de los parámetros de la red neuronal usando un batch de estados y batch de vectores de valores
        de la función estado-acción correspondientes
        """
        self.model.fit(states, q_values, verbose=0)
    
    def predict(self, s):
        """
        Realiza una predicción de la función de valor estado-accion dado el estado
        
        Argumentos:
            s: estado para el cual realizar la predicción
            
        Retorna:
            Un vector con la predicción de la función de valor para todas las accións
        """
        return self.model.predict(self.process_state_for_network(s))
    
    def process_state_for_network(self, state):
        """Scale, convert to greyscale and store as float32.
        Basically same as process state for memory, but this time
        outputs float32 images.
        """
        state = state.astype('float')
        return state

class Memory():
    def __init__(self, memory_size=100000, history_length=4):
        self.history = np.zeros((84, 84, history_length))
        self.history_length = history_length
        self.memory = deque(maxlen=memory_size)
    
    def remember(self, state, action, reward, next_state, done):
        """
        Guardar una tupla de estado, accion, recompensa, proximo estado, estado_terminal.
        """
        self.memory.append((state, action, reward, next_state, done))
        
    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)
    
    def preprocess_image(self, state):
        """
        Convertir la imagen a blanco y negro y reducirla a 84x84.
        """
        I = Image.fromarray(state, 'RGB')
        I = I.convert('L')  # to gray
        I = I.resize((84, 84), Image.ANTIALIAS)
        I = np.array(I).astype('uint8')
        return I
    
    def push_frame(self, state):
        """
        Pushear el frame a la stack de frames.
        """
        state = self.preprocess_image(state)
        self.history[..., 0] = state
        self.history = np.roll(self.history, -1, axis=-1)
        return self.history.copy()
    
    def reset_history(self):
        """
        Resetear el stack de frames.
        """
        self.history = np.zeros((84, 84, self.history_length))
    
    def __len__(self):
        return len(self.memory)

In [ ]:
class Learner:
    def __init__(self, env, estimator, estimator_target, num_steps, discount_factor=1.0, 
                 exploration_max=1.0, exploration_min=0.05, epsilon_decay=0.99, memory_size=100000, 
                 batch_size=64, target_update_freq=1000, history_length=4, train_freq=4, burnin=100000,
                 save_freq = 50000):
        """
        Algoritmo q-learning/sarsa con experience replay utilizando aproximación de funciones.

        Argumentos de inicialización:
            env: ambiente de OpenAI.
            estimator: función de aproximación de la función de valor estado-acción.
            estimator_target: función de aproximación target de la función de valor estado-acción.
            num_steps: número de pasos durante los cuales entrenar el estimador.
            discount_factor: factor de descuento gama.
            exploration_max: probabilidad de tomar una acción aleatoria inicialmente.
            exploration_min: mínima probabilidad de tomar una acción aleatoria.
            epsilon_decay: luego de cada episodio la probabilidad de una acción aleatoria decae por este factor.
            target_update_freq: frecuencia de update de la función de aproximación target.
            train_freq: frecuencia de entrenamiento de la función de aproximación de la función de valor.
            history_length: cantidad de frames que se mandan a la red neuronal.
            burnin: cantidad inicial de pasos exploratorios para tener la memoria inicializada.
            save_freq: cada cuántos pasos se guardan los modelos.
        """
        self.env = env
        self.discount_factor = discount_factor
        
        self.estimator = estimator
        self.estimator_target = estimator_target
        
        self.num_steps = num_steps
        
        self.exploration_rate = exploration_max
        self.exploration_min = exploration_min
        self.epsilon_decay = epsilon_decay
        self.train_freq = train_freq
        self.batch_size = batch_size
        self.target_update_freq = target_update_freq
        
        self.memory = Memory(memory_size, history_length)
        
        self.history_length = history_length
        
        self.burnin = burnin
        self.save_freq = save_freq
        
        
    def decay_exploration_rate(self):
        # decae la probabilidad de exploración
        self.exploration_rate *= self.epsilon_decay
        self.exploration_rate = max(self.exploration_min, self.exploration_rate)
    
    def policy_fn(self, state, epsilon):
        """
        Política epsilon-greedy basada en la función de aproximación actual y la probabilidad de exploración epsilon.

        Retorna:
            Un vector con la probabilidad de tomar cada acción.
        """
        A = np.ones(env.action_space.n, dtype=float) * self.exploration_rate / env.action_space.n
        q_values = self.estimator.predict(state)
        best_action = np.argmax(q_values[0])
        A[best_action] += (1.0 - epsilon)
        return A
    
    def experience_replay(self):
        """
        Realiza una corrida de entrenamiento de la función de aproximación dado un batch de experiencia acumulada.
        """
        if len(self.memory) < self.batch_size:
            return
        # samplear de la memoria
        batch = self.memory.sample(self.batch_size)
        
        batch_q_values = np.zeros((self.batch_size, self.env.action_space.n))
        batch_states = np.zeros((self.batch_size, 84, 84, self.history_length))
        
        for i_batch, (state, action, reward, next_state, terminal) in enumerate(batch):
            
            q_update = reward
            if not terminal:
                ## el update de q-learning para este ejemplo
                q_update = reward + self.discount_factor * np.amax(self.estimator_target.predict(next_state)[0])
            
            q_values = self.estimator.predict(state)
            q_values[0][action] = q_update
            batch_q_values[i_batch] = q_values
            batch_states[i_batch] = state
            i_batch += 1
        
        # un paso en la dirección del gradiente
        self.estimator.update(batch_states, batch_q_values, verbose=0)
        
    def train(self):    
        """
        Realiza Q-Learning o Sarsa con aproximación de la función de valor estado-acción.
        Retorna:
            Un objeto de tipo EpisodeStats con dos arrays de numpy para la longitud y recompensa acumulada de cada
            episodio respectivamente.
        """
        stats = plotting.EpisodeStats(
            episode_lengths=[],
            episode_rewards=[])    
        
        # realizamos self.burnin pasos iniciales de exploración para inicializar la memoria
        itr = 0
        while itr <= self.burnin:
            step = 0
            self.memory.reset_history()
            state = env.reset()
            # guardar en el stack de frames
            state = self.memory.push_frame(state)
            # hacer un reshape del estado para poder pasarlo por la red neronal
            state = np.reshape(state, [1, 84, 84, self.history_length])
            # resetear el reward
            cum_reward = 0.0
            # elegir acción al azar
            action = np.random.choice(self.env.action_space.n)
            while True:
                env.render()
                # tomar la acción elegida
                next_state, reward, terminal, info = env.step(action)
                cum_reward += reward*self.discount_factor**step
                # guardar en el stack de frames
                next_state = self.memory.push_frame(next_state)
                itr += 1; step += 1
                # hacer un reshape del estado para poder pasarlo por la red neronal
                next_state = np.reshape(next_state, [1, 84, 84, self.history_length])
                # elegir la próxima acción al azar
                if not terminal:
                    next_action = np.random.choice(self.env.action_space.n)
                else:
                    next_action = None
                    self.memory.reset_history()
                # almacenar en la memoria
                self.memory.remember(state, action, reward, next_state, terminal)
                # actualizar estado y acción
                state = next_state; action = next_action
                if terminal:
                    print("Iteración de burnin:" + str(itr) + 
                          ", Exploración: " + str(self.exploration_rate) + 
                          ", Recompensa Acumulada: " + str(cum_reward))
                    break
        
        # comienza el loop de entrenamiento
        itr = 0
        step = 0
        n_episode = 0 
        while itr <=  self.num_steps:
            # resetear el stack de frames (nuevo episodio)
            self.memory.reset_history()
            state = env.reset()
            # guardar en el stack de frames
            state = self.memory.push_frame(state)
            # hacer un reshape del estado para poder pasarlo por la red neronal
            state = np.reshape(state, [1, 84, 84, self.history_length])
            cum_reward = 0.0    
            # elegir la acción
            action = np.random.choice(self.env.action_space.n, p=self.policy_fn(state, self.exploration_rate))
            while True:
                env.render()
                # realizar un paso con la acción elegida
                next_state, reward, terminal, info = env.step(action)
                # acumular la recompensa
                cum_reward += reward*self.discount_factor**step
                # guardar en el stack de frames
                next_state = self.memory.push_frame(next_state)
                itr += 1; step += 1
                # hacer un reshape del estado para poder pasarlo por la red nueronal
                next_state = np.reshape(next_state, [1, 84, 84, self.history_length])
                if not terminal:
                    # elegir la próxima acción
                    next_action = np.random.choice(self.env.action_space.n, p=self.policy_fn(next_state, self.exploration_rate))
                else:
                    next_action = None
                    self.memory.reset_history()
                
                self.memory.remember(state, action, reward, next_state, terminal)
                # actualizar el estado y la acción
                state = next_state; action = next_action
                if terminal:
                    print("Episodio: " + str(n_episode) + 
                          ", Iteración:" + str(itr) + 
                          ", Exploración: " + str(self.exploration_rate) + 
                          ", Recompensa Acumulada: " + str(cum_reward))
                    # Actualizar estadísticas
                    stats.episode_rewards.append(cum_reward)
                    stats.episode_lengths.append(step)
                    break        
                    
                # copiar los pesos de la red siendo entrenada al target
                if itr % self.target_update_freq == 0:
                    self.estimator_target.model.set_weights(self.estimator.model.get_weights())
                # entrenar la red cada train_freq steps
                if itr % self.train_freq == 0:
                    self.experience_replay()
                # guardar la red nueronal
                if itr % self.save_freq == 0:
                    self.estimator.model.save("model_" + str(itr) + ".h5")
            # decaer la probabilidad de exploración
            self.decay_exploration_rate()
            n_episode += 1
        return stats

In [ ]:
estimator = Estimator(env)
estimator_target = Estimator(env)
learner = Learner(env, estimator, estimator_target, 1000000)

In [ ]:
stats = learner.train()

In [ ]:
estimator = Estimator(env)
estimator.model = keras.models.load_model("model_450000.h5")
estimator_target = Estimator(env)
estimator_target.model = keras.models.load_model("model_450000.h5")
learner = Learner(env, estimator, estimator_target, 500000, discount_factor=1.0, exploration_max=0.2,  xploration_min=0.05,
                  epsilon_decay=0.99, burnin=100000, save_freq = 100000)